# Hoja de Trabajo 2

Gabriel García - 21352        
Luis Montenegro - 21699

# Ejercicio 1 - Experimentación Practica

En esta actividad, implementará y comparará diferentes funciones de pérdida y técnicas de regularización utilizando PyTorch. Utilizará el conjunto de datos de Iris para una tarea de clasificación y una arquitectura básica de red neuronal de feedforward. El objetivo es observar cómo las diferentes opciones impactan la convergencia y el rendimiento del modelo.

In [1]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
import numpy as np

# Task 1 - Preparación del conjunto de datos

Cargue el conjunto de datos de Iris utilizando bibliotecas como sklearn.datasets. Luego, divida el conjunto de datos en conjuntos de entrenamiento y validación

In [2]:
# Cargar el dataset de iris
iris = datasets.load_iris()
X = iris.data
y = iris.target

In [3]:
# Escalar los datos
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [4]:
# Dividir el conjunto de datos en conjuntos de entrenamiento y validación (80% entrenamiento, 20% validación)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [5]:
# Convertir los datos a tensores de PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

In [6]:
# Crear conjuntos de datos y cargadores de datos
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [7]:
# Imprimir las formas de los conjuntos de datos para verificación
print(f"Tamaño del conjunto de entrenamiento: {len(train_loader.dataset)} muestras")
print(f"Tamaño del conjunto de validación: {len(val_loader.dataset)} muestras")

Tamaño del conjunto de entrenamiento: 120 muestras
Tamaño del conjunto de validación: 30 muestras


# Task 2 - Arquitectura modelo
Cree una red neuronal feedforward simple utilizando nn.Module de PyTorch. Luego, defina capa de entrada, capas ocultas y capa de salida. Después, elija las funciones de activación y el número de neuronas por capa.

In [8]:
import torch.nn as nn
import torch.nn.functional as F

In [9]:
# Definir la arquitectura de la red neuronal
class SimpleFeedforwardNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(SimpleFeedforwardNN, self).__init__()
        # Definir la capa de entrada
        self.fc1 = nn.Linear(input_size, hidden_size1)
        # Definir la primera capa oculta
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        # Definir la capa de salida
        self.fc3 = nn.Linear(hidden_size2, output_size)
        
    def forward(self, x):
        # Aplicar activación ReLU a la capa de entrada
        x = F.relu(self.fc1(x))
        # Aplicar activación ReLU a la primera capa oculta
        x = F.relu(self.fc2(x))
        # Aplicar activación Softmax a la capa de salida para obtener probabilidades
        x = F.log_softmax(self.fc3(x), dim=1)
        return x

In [10]:
# Parámetros del modelo
input_size = 4  # El conjunto de datos de Iris tiene 4 características de entrada
hidden_size1 = 10  # Número de neuronas en la primera capa oculta
hidden_size2 = 8   # Número de neuronas en la segunda capa oculta
output_size = 3    # El conjunto de datos de Iris tiene 3 clases de salida

# Instanciar el modelo
model = SimpleFeedforwardNN(input_size, hidden_size1, hidden_size2, output_size)

# Mostrar la arquitectura del modelo
print(model)

SimpleFeedforwardNN(
  (fc1): Linear(in_features=4, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=8, bias=True)
  (fc3): Linear(in_features=8, out_features=3, bias=True)
)


# Task 3 - Funciones de Pérdida
Utilice diferentes funciones de pérdida comunes como Cross-Entropy Loss y MSE para clasificación. Entrene el modelo con diferentes funciones de pérdida y registre las pérdidas de entrenamiento y test. Debe utilizar al menos 3 diferentes funciones. Es decir, procure que su código sea capaz de parametrizar el uso de diferentes funciones de pérdida

In [11]:
import torch.optim as optim

In [17]:
import torch

def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=20):
    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            
            # Ajuste para CrossEntropyLoss y NLLLoss
            if isinstance(criterion, (nn.CrossEntropyLoss, nn.NLLLoss)):
                # Las etiquetas deben ser de tipo Long para estas funciones
                labels = labels.long()
            
            # Ajuste para MSELoss: convertir labels a one-hot encoding
            elif isinstance(criterion, nn.MSELoss):
                labels = F.one_hot(labels, num_classes=output_size).float()

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        
        epoch_loss = running_loss / len(train_loader.dataset)
        train_losses.append(epoch_loss)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                
                # Ajuste para CrossEntropyLoss y NLLLoss
                if isinstance(criterion, (nn.CrossEntropyLoss, nn.NLLLoss)):
                    labels = labels.long()
                
                # Ajuste para MSELoss: convertir labels a one-hot encoding
                elif isinstance(criterion, nn.MSELoss):
                    labels = F.one_hot(labels, num_classes=output_size).float()

                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)

        val_loss /= len(val_loader.dataset)
        val_losses.append(val_loss)
        
        print(f'Epoch {epoch+1}/{epochs}, Training Loss: {epoch_loss:.4f}, Validation Loss: {val_loss:.4f}')

    return train_losses, val_losses


In [18]:
# Instanciar el modelo y optimizador
model = SimpleFeedforwardNN(input_size, hidden_size1, hidden_size2, output_size)

In [19]:
# Definir el optimizador
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [20]:
# Definir las funciones de pérdida
loss_functions = {
    'CrossEntropyLoss': nn.CrossEntropyLoss(),
    'MSELoss': nn.MSELoss(),
    'NLLLoss': nn.NLLLoss()
}

In [21]:
# Entrenar y registrar pérdidas para cada función de pérdida
results = {}
for loss_name, criterion in loss_functions.items():
    print(f'\nEntrenando con {loss_name}...')
    model = SimpleFeedforwardNN(input_size, hidden_size1, hidden_size2, output_size)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    train_losses, val_losses = train_model(model, train_loader, val_loader, criterion, optimizer, epochs=20)
    results[loss_name] = {'train_loss': train_losses, 'val_loss': val_losses}



Entrenando con CrossEntropyLoss...
Epoch 1/20, Training Loss: 1.0529, Validation Loss: 1.0444
Epoch 2/20, Training Loss: 1.0391, Validation Loss: 1.0309
Epoch 3/20, Training Loss: 1.0264, Validation Loss: 1.0171
Epoch 4/20, Training Loss: 1.0123, Validation Loss: 1.0037
Epoch 5/20, Training Loss: 0.9989, Validation Loss: 0.9899
Epoch 6/20, Training Loss: 0.9844, Validation Loss: 0.9754
Epoch 7/20, Training Loss: 0.9692, Validation Loss: 0.9596
Epoch 8/20, Training Loss: 0.9523, Validation Loss: 0.9424
Epoch 9/20, Training Loss: 0.9344, Validation Loss: 0.9240
Epoch 10/20, Training Loss: 0.9150, Validation Loss: 0.9042
Epoch 11/20, Training Loss: 0.8949, Validation Loss: 0.8829
Epoch 12/20, Training Loss: 0.8722, Validation Loss: 0.8606
Epoch 13/20, Training Loss: 0.8492, Validation Loss: 0.8368
Epoch 14/20, Training Loss: 0.8242, Validation Loss: 0.8118
Epoch 15/20, Training Loss: 0.7978, Validation Loss: 0.7853
Epoch 16/20, Training Loss: 0.7708, Validation Loss: 0.7583
Epoch 17/20, 

# Task 4 - Técnicas de Regularización
Utilice distintas técnicas de regularización como L1, L2 y dropout. Entrene el modelo con y sin técnicas de regularización y observe el impacto en el overfitting y la generalización. Debe utilizar al menos 3 diferentes técnicas. Es decir, procure que su código sea capaz de parametrizar el uso de diferentes técnicas de regularización